In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
     
batch_size = 128
model_name = "Resnet20_quantAware"
model = resnet20_quant() # For quantization aware training

# print(model)


normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))     

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
Files already downloaded and verified
Files already downloaded and verified


In [2]:
def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [20, 30, 150, 225]
    p_adjust_list = [25, 35]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
    if epoch in p_adjust_list:
        for param_group in optimizer.param_groups:
            param_group['momentum'] = param_group['momentum'] / 2

In [3]:
# This cell won't be given, but students will complete the training

lr = 0.1
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 2.177 (2.177)	Data 0.378 (0.378)	Loss 2.4776 (2.4776)	Prec 10.938% (10.938%)
Epoch: [0][100/391]	Time 0.041 (0.067)	Data 0.002 (0.006)	Loss 1.6168 (1.9811)	Prec 39.062% (25.186%)
Epoch: [0][200/391]	Time 0.037 (0.056)	Data 0.002 (0.004)	Loss 1.7641 (1.8235)	Prec 31.250% (31.110%)
Epoch: [0][300/391]	Time 0.051 (0.052)	Data 0.002 (0.004)	Loss 1.3537 (1.7224)	Prec 50.000% (34.972%)
Validation starts
Test: [0/79]	Time 0.243 (0.243)	Loss 1.3392 (1.3392)	Prec 56.250% (56.250%)
 * Prec 51.000% 
best acc: 51.000000
Epoch: [1][0/391]	Time 0.488 (0.488)	Data 0.431 (0.431)	Loss 1.2776 (1.2776)	Prec 51.562% (51.562%)
Epoch: [1][100/391]	Time 0.038 (0.051)	Data 0.002 (0.007)	Loss 1.2367 (1.2494)	Prec 57.031% (54.680%)
Epoch: [1][200/391]	Time 0.044 (0.048)	Data 0.002 (0.005)	Loss 1.2049 (1.1962)	Prec 57.031% (56.736%)
Epoch: [1][300/391]	Time 0.039 (0.046)	Data 0.002 (0.004)	Loss 1.0800 (1.1602)	Prec 61.719% (58.153%)
Validation starts
Test: [0/79]	Time 0.452 (0.452)	Loss 1.

Epoch: [15][200/391]	Time 0.039 (0.050)	Data 0.002 (0.005)	Loss 0.3969 (0.4075)	Prec 85.938% (85.766%)
Epoch: [15][300/391]	Time 0.042 (0.048)	Data 0.002 (0.004)	Loss 0.3101 (0.4069)	Prec 85.156% (85.922%)
Validation starts
Test: [0/79]	Time 0.251 (0.251)	Loss 0.4592 (0.4592)	Prec 82.031% (82.031%)
 * Prec 81.250% 
best acc: 81.740000
Epoch: [16][0/391]	Time 0.621 (0.621)	Data 0.559 (0.559)	Loss 0.3243 (0.3243)	Prec 88.281% (88.281%)
Epoch: [16][100/391]	Time 0.034 (0.050)	Data 0.002 (0.008)	Loss 0.4085 (0.3775)	Prec 85.938% (86.680%)
Epoch: [16][200/391]	Time 0.047 (0.045)	Data 0.002 (0.005)	Loss 0.3601 (0.3861)	Prec 87.500% (86.419%)
Epoch: [16][300/391]	Time 0.042 (0.044)	Data 0.002 (0.004)	Loss 0.3129 (0.3925)	Prec 88.281% (86.176%)
Validation starts
Test: [0/79]	Time 0.256 (0.256)	Loss 0.4277 (0.4277)	Prec 85.156% (85.156%)
 * Prec 83.290% 
best acc: 83.290000
Epoch: [17][0/391]	Time 0.625 (0.625)	Data 0.567 (0.567)	Loss 0.3780 (0.3780)	Prec 87.500% (87.500%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.037 (0.045)	Data 0.002 (0.004)	Loss 0.1360 (0.2025)	Prec 95.312% (92.951%)
Validation starts
Test: [0/79]	Time 0.246 (0.246)	Loss 0.2292 (0.2292)	Prec 93.750% (93.750%)
 * Prec 89.050% 
best acc: 89.150000
Epoch: [31][0/391]	Time 0.525 (0.525)	Data 0.463 (0.463)	Loss 0.2170 (0.2170)	Prec 93.750% (93.750%)
Epoch: [31][100/391]	Time 0.043 (0.047)	Data 0.002 (0.007)	Loss 0.1706 (0.2033)	Prec 94.531% (92.930%)
Epoch: [31][200/391]	Time 0.043 (0.046)	Data 0.002 (0.005)	Loss 0.1137 (0.2013)	Prec 99.219% (92.996%)
Epoch: [31][300/391]	Time 0.041 (0.045)	Data 0.002 (0.004)	Loss 0.2414 (0.2012)	Prec 92.969% (92.987%)
Validation starts
Test: [0/79]	Time 0.373 (0.373)	Loss 0.2429 (0.2429)	Prec 92.188% (92.188%)
 * Prec 89.080% 
best acc: 89.150000
Epoch: [32][0/391]	Time 0.470 (0.470)	Data 0.407 (0.407)	Loss 0.1636 (0.1636)	Prec 94.531% (94.531%)
Epoch: [32][100/391]	Time 0.037 (0.048)	Data 0.002 (0.006)	Loss 0.2625 (0.2042)	Prec 86.719% (93.062%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.240 (0.240)	Loss 0.2320 (0.2320)	Prec 94.531% (94.531%)
 * Prec 89.080% 
best acc: 89.150000
Epoch: [46][0/391]	Time 0.510 (0.510)	Data 0.450 (0.450)	Loss 0.1221 (0.1221)	Prec 96.875% (96.875%)
Epoch: [46][100/391]	Time 0.044 (0.047)	Data 0.002 (0.007)	Loss 0.1875 (0.1979)	Prec 92.188% (93.085%)
Epoch: [46][200/391]	Time 0.042 (0.046)	Data 0.002 (0.004)	Loss 0.2127 (0.1962)	Prec 92.969% (93.085%)
Epoch: [46][300/391]	Time 0.037 (0.046)	Data 0.001 (0.004)	Loss 0.1179 (0.1976)	Prec 96.875% (93.023%)
Validation starts
Test: [0/79]	Time 0.400 (0.400)	Loss 0.2683 (0.2683)	Prec 90.625% (90.625%)
 * Prec 88.960% 
best acc: 89.150000
Epoch: [47][0/391]	Time 0.473 (0.473)	Data 0.412 (0.412)	Loss 0.2052 (0.2052)	Prec 91.406% (91.406%)
Epoch: [47][100/391]	Time 0.050 (0.048)	Data 0.002 (0.006)	Loss 0.2364 (0.1989)	Prec 92.969% (92.992%)
Epoch: [47][200/391]	Time 0.055 (0.046)	Data 0.003 (0.004)	Loss 0.1592 (0.1966)	Prec 94.531% (93.136%)
Epoch: [47][300/391]	

 * Prec 89.020% 
best acc: 89.330000
Epoch: [61][0/391]	Time 0.702 (0.702)	Data 0.639 (0.639)	Loss 0.1719 (0.1719)	Prec 92.969% (92.969%)
Epoch: [61][100/391]	Time 0.042 (0.051)	Data 0.002 (0.009)	Loss 0.1411 (0.2000)	Prec 95.312% (93.046%)
Epoch: [61][200/391]	Time 0.040 (0.047)	Data 0.002 (0.006)	Loss 0.1935 (0.1979)	Prec 89.844% (93.140%)
Epoch: [61][300/391]	Time 0.039 (0.046)	Data 0.002 (0.004)	Loss 0.1662 (0.1955)	Prec 92.969% (93.132%)
Validation starts
Test: [0/79]	Time 0.266 (0.266)	Loss 0.2362 (0.2362)	Prec 92.188% (92.188%)
 * Prec 89.160% 
best acc: 89.330000
Epoch: [62][0/391]	Time 0.578 (0.578)	Data 0.518 (0.518)	Loss 0.1408 (0.1408)	Prec 96.094% (96.094%)
Epoch: [62][100/391]	Time 0.049 (0.053)	Data 0.002 (0.008)	Loss 0.0957 (0.1974)	Prec 96.875% (93.100%)
Epoch: [62][200/391]	Time 0.054 (0.051)	Data 0.003 (0.005)	Loss 0.1277 (0.1961)	Prec 95.312% (93.241%)
Epoch: [62][300/391]	Time 0.043 (0.047)	Data 0.002 (0.004)	Loss 0.1642 (0.1949)	Prec 92.969% (93.293%)
Validation s

Epoch: [76][100/391]	Time 0.044 (0.050)	Data 0.003 (0.007)	Loss 0.2217 (0.1961)	Prec 90.625% (92.953%)
Epoch: [76][200/391]	Time 0.036 (0.047)	Data 0.002 (0.005)	Loss 0.2258 (0.1941)	Prec 89.062% (93.163%)
Epoch: [76][300/391]	Time 0.050 (0.046)	Data 0.002 (0.004)	Loss 0.1393 (0.1951)	Prec 96.094% (93.122%)
Validation starts
Test: [0/79]	Time 0.456 (0.456)	Loss 0.2431 (0.2431)	Prec 92.969% (92.969%)
 * Prec 89.010% 
best acc: 89.330000
Epoch: [77][0/391]	Time 0.649 (0.649)	Data 0.590 (0.590)	Loss 0.2186 (0.2186)	Prec 92.969% (92.969%)
Epoch: [77][100/391]	Time 0.031 (0.049)	Data 0.002 (0.008)	Loss 0.1698 (0.2020)	Prec 94.531% (93.000%)
Epoch: [77][200/391]	Time 0.045 (0.046)	Data 0.002 (0.005)	Loss 0.1978 (0.2000)	Prec 94.531% (93.085%)
Epoch: [77][300/391]	Time 0.036 (0.046)	Data 0.001 (0.004)	Loss 0.1943 (0.1957)	Prec 94.531% (93.278%)
Validation starts
Test: [0/79]	Time 0.500 (0.500)	Loss 0.2395 (0.2395)	Prec 91.406% (91.406%)
 * Prec 88.900% 
best acc: 89.330000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.036 (0.048)	Data 0.002 (0.005)	Loss 0.3505 (0.1895)	Prec 88.281% (93.455%)
Epoch: [91][300/391]	Time 0.049 (0.047)	Data 0.002 (0.004)	Loss 0.1606 (0.1912)	Prec 95.312% (93.452%)
Validation starts
Test: [0/79]	Time 0.466 (0.466)	Loss 0.2534 (0.2534)	Prec 90.625% (90.625%)
 * Prec 89.030% 
best acc: 89.330000
Epoch: [92][0/391]	Time 0.575 (0.575)	Data 0.516 (0.516)	Loss 0.1611 (0.1611)	Prec 92.969% (92.969%)
Epoch: [92][100/391]	Time 0.049 (0.049)	Data 0.002 (0.008)	Loss 0.2464 (0.1930)	Prec 92.188% (93.379%)
Epoch: [92][200/391]	Time 0.028 (0.047)	Data 0.002 (0.005)	Loss 0.1950 (0.1903)	Prec 92.969% (93.256%)
Epoch: [92][300/391]	Time 0.057 (0.046)	Data 0.002 (0.004)	Loss 0.1460 (0.1898)	Prec 93.750% (93.330%)
Validation starts
Test: [0/79]	Time 0.405 (0.405)	Loss 0.2522 (0.2522)	Prec 92.969% (92.969%)
 * Prec 88.980% 
best acc: 89.330000
Epoch: [93][0/391]	Time 0.544 (0.544)	Data 0.500 (0.500)	Loss 0.1896 (0.1896)	Prec 92.188% (92.188%)
Epoch: [93][100/391]	

In [4]:
PATH = "result/" + model_name + "/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8933/10000 (89%)



In [7]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  

In [6]:
######### Save inputs from selected layer ##########
save_output = SaveOutput()
model.layer1[0].conv2.register_forward_pre_hook(save_output) ## Grab input for BasicBlock0 Conv1 layer
# model.layer1[1].conv1.register_forward_pre_hook(save_output) ## Grab input for BasicBlock1 Conv1 layer
####################################################

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to('cuda')
# Run a forward pass to prehook
out = model(images)

In [7]:
w_bit = 4
weight_q = model.layer1[0].conv2.weight_q # quantized value is stored during the training
# print(weight_q)
w_alpha = model.layer1[0].conv2.weight_quant.wgt_alpha   # alpha is defined in your model already. bring it out here
w_delta = w_alpha/(2**(w_bit-1)-1)   # delta can be calculated by using alpha and w_bit
weight_int = weight_q / w_delta # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-2.0000, -4.0000,  7.0000],
          [ 2.0000, -7.0000, -1.0000],
          [-5.0000, -2.0000,  5.0000]],

         [[ 2.0000,  1.0000,  1.0000],
          [ 2.0000, -3.0000, -4.0000],
          [ 0.0000, -2.0000, -4.0000]],

         [[ 6.0000,  3.0000, -6.0000],
          [-6.0000,  7.0000,  4.0000],
          [-4.0000,  4.0000, -4.0000]],

         ...,

         [[ 3.0000, -2.0000, -3.0000],
          [ 3.0000, -1.0000, -1.0000],
          [ 2.0000, -2.0000, -2.0000]],

         [[ 1.0000, -5.0000,  3.0000],
          [ 1.0000, -7.0000,  7.0000],
          [ 2.0000, -7.0000,  5.0000]],

         [[-4.0000, -4.0000, -7.0000],
          [-4.0000,  3.0000, -6.0000],
          [-1.0000,  1.0000, -2.0000]]],


        [[[-1.0000, -0.0000,  2.0000],
          [ 3.0000, -1.0000,  4.0000],
          [ 2.0000,  2.0000,  3.0000]],

         [[-3.0000, -0.0000,  1.0000],
          [ 1.0000,  3.0000,  4.0000],
          [ 2.0000,  3.0000,  6.0000]],

         [[-2.0000, -4.0000, -3

In [8]:
x_bit = 4    
x = save_output.outputs[0][0]  # input of the 2nd conv layer
x_alpha = model.layer1[0].conv2.act_alpha
x_delta = x_alpha/(2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  4.0000,  4.0000,  0.0000],
          [ 3.0000,  2.0000,  2.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  0.0000,  0.0000,  ...,  3.0000,  1.0000,  0.0000],
          ...,
          [ 0.0000,  1.0000,  2.0000,  ...,  3.0000,  1.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 2.0000,  3.0000,  3.0000,  ...,  3.0000,  1.0000,  0.0000]],

         [[ 2.0000,  1.0000,  1.0000,  ...,  0.0000,  1.0000,  0.0000],
          [ 4.0000,  2.0000,  2.0000,  ...,  0.0000,  1.0000,  0.0000],
          [ 4.0000,  2.0000,  1.0000,  ...,  0.0000,  1.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  1.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  1.0000,  0.0000]],

         [[ 3.0000,  2.0000,  4.0000,  ...,  3.0000,  2.0000,  3.0000],
          [ 1.0000,  1.0000,  

In [9]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_recovered = output_int*x_delta*w_delta  # recover with x_delta and w_delta
print(output_recovered)

tensor([[[[ 1.1223e+01,  2.3106e+00,  9.0775e-01,  ...,  5.1329e+01,
           -1.0976e+01,  2.3932e+00],
          [ 1.1388e+01,  5.2815e+00,  1.1553e+00,  ...,  4.4975e+01,
           -1.6340e+01, -6.3543e+00],
          [ 2.9708e+00,  3.9350e-08,  6.3543e+00,  ...,  4.8689e+01,
           -1.8485e+01, -4.9514e+00],
          ...,
          [ 5.9417e+00, -4.5388e+00, -5.1164e+00,  ...,  8.0047e+00,
           -5.3640e+00, -3.1359e+00],
          [ 6.1892e+00,  7.2620e+00,  8.9950e+00,  ...,  6.1067e+00,
            3.8786e+00,  9.0775e-01],
          [ 2.1456e+00,  2.6407e+00,  5.2815e+00,  ...,  3.7961e+00,
           -1.8980e+00, -5.1990e+00]],

         [[ 1.6587e+01,  1.1058e+01,  7.5096e+00,  ..., -9.3251e+00,
           -6.1892e+00,  5.4465e+00],
          [ 1.7908e+01,  1.3534e+01,  1.2791e+01,  ..., -5.3640e+00,
           -2.3932e+00,  8.0873e+00],
          [ 7.9222e+00,  9.1601e+00,  1.2874e+01,  ..., -4.4562e+00,
           -1.7330e+00,  9.0775e+00],
          ...,
     

In [10]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv2.weight_q
print(conv_ref.weight)
output_ref = conv_ref(x)
print(output_ref)

Parameter containing:
tensor([[[[-0.6599, -1.3198,  2.3097],
          [ 0.6599, -2.3097, -0.3300],
          [-1.6498, -0.6599,  1.6498]],

         [[ 0.6599,  0.3300,  0.3300],
          [ 0.6599, -0.9899, -1.3198],
          [ 0.0000, -0.6599, -1.3198]],

         [[ 1.9797,  0.9899, -1.9797],
          [-1.9797,  2.3097,  1.3198],
          [-1.3198,  1.3198, -1.3198]],

         ...,

         [[ 0.9899, -0.6599, -0.9899],
          [ 0.9899, -0.3300, -0.3300],
          [ 0.6599, -0.6599, -0.6599]],

         [[ 0.3300, -1.6498,  0.9899],
          [ 0.3300, -2.3097,  2.3097],
          [ 0.6599, -2.3097,  1.6498]],

         [[-1.3198, -1.3198, -2.3097],
          [-1.3198,  0.9899, -1.9797],
          [-0.3300,  0.3300, -0.6599]]],


        [[[-0.3300, -0.0000,  0.6599],
          [ 0.9899, -0.3300,  1.3198],
          [ 0.6599,  0.6599,  0.9899]],

         [[-0.9899, -0.0000,  0.3300],
          [ 0.3300,  0.9899,  1.3198],
          [ 0.6599,  0.9899,  1.9797]],

         

In [11]:
difference = abs( output_ref - output_recovered )
# diff_percent = 100*abs(output_ref - output_recovered)/output_ref
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(0.6073, device='cuda:0', grad_fn=<MeanBackward0>)


In [12]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
weight = model.layer1[0].conv2.weight_q
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


tensor([[[[ 1.0943e+01,  2.2941e+00,  6.8174e-01,  ...,  5.2347e+01,
           -1.0455e+01,  3.5751e+00],
          [ 1.1056e+01,  5.2068e+00,  1.7795e+00,  ...,  4.6503e+01,
           -1.7367e+01, -4.7495e+00],
          [ 3.2162e+00,  8.9087e-01,  6.5367e+00,  ...,  5.0049e+01,
           -1.7903e+01, -3.8221e+00],
          ...,
          [ 5.9307e+00, -4.3173e+00, -4.2835e+00,  ...,  7.0285e+00,
           -6.0234e+00, -1.7933e+00],
          [ 5.2925e+00,  9.1368e+00,  9.3217e+00,  ...,  6.3085e+00,
            3.4013e+00,  2.8006e+00],
          [ 1.4867e+00,  2.6702e+00,  4.5704e+00,  ...,  2.9954e+00,
           -1.9712e+00, -4.2182e+00]],

         [[ 1.6718e+01,  1.1372e+01,  8.4392e+00,  ..., -8.7674e+00,
           -5.5232e+00,  5.1990e+00],
          [ 1.8254e+01,  1.3352e+01,  1.2206e+01,  ..., -5.5794e+00,
           -1.8672e+00,  8.4840e+00],
          [ 9.3280e+00,  9.2767e+00,  1.2451e+01,  ..., -4.3460e+00,
           -1.2451e+00,  9.6297e+00],
          ...,
     

In [13]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(0.6450, device='cuda:0', grad_fn=<MeanBackward0>)


In [16]:
# Start process for 2bit

model_name = "Resnet20_quantAware_2bit"
model = resnet20_quant_2bit() # For quantization aware training

# Train the model
lr = 0.1
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.322 (0.322)	Data 0.236 (0.236)	Loss 2.4225 (2.4225)	Prec 5.469% (5.469%)
Epoch: [0][100/391]	Time 0.053 (0.058)	Data 0.003 (0.007)	Loss 2.0943 (2.2490)	Prec 19.531% (16.801%)
Epoch: [0][200/391]	Time 0.047 (0.055)	Data 0.002 (0.004)	Loss 2.0124 (2.1322)	Prec 26.562% (20.394%)
Epoch: [0][300/391]	Time 0.047 (0.054)	Data 0.002 (0.004)	Loss 2.0876 (2.1043)	Prec 25.781% (21.501%)
Validation starts
Test: [0/79]	Time 0.284 (0.284)	Loss 2.0196 (2.0196)	Prec 28.125% (28.125%)
 * Prec 27.500% 
best acc: 27.500000
Epoch: [1][0/391]	Time 0.301 (0.301)	Data 0.244 (0.244)	Loss 1.9146 (1.9146)	Prec 27.344% (27.344%)
Epoch: [1][100/391]	Time 0.052 (0.060)	Data 0.002 (0.005)	Loss 1.8568 (1.8683)	Prec 35.156% (30.391%)
Epoch: [1][200/391]	Time 0.071 (0.060)	Data 0.003 (0.004)	Loss 1.7896 (1.8510)	Prec 39.062% (31.370%)
Epoch: [1][300/391]	Time 0.059 (0.059)	Data 0.003 (0.003)	Loss 1.8257 (1.8303)	Prec 30.469% (32.036%)
Validation starts
Test: [0/79]	Time 0.256 (0.256)	Loss 1.78

Epoch: [15][200/391]	Time 0.046 (0.061)	Data 0.002 (0.003)	Loss 0.9636 (1.0007)	Prec 68.750% (64.428%)
Epoch: [15][300/391]	Time 0.047 (0.058)	Data 0.002 (0.003)	Loss 0.9902 (0.9946)	Prec 60.938% (64.745%)
Validation starts
Test: [0/79]	Time 0.238 (0.238)	Loss 1.0765 (1.0765)	Prec 64.062% (64.062%)
 * Prec 60.200% 
best acc: 62.740000
Epoch: [16][0/391]	Time 0.315 (0.315)	Data 0.250 (0.250)	Loss 0.9053 (0.9053)	Prec 73.438% (73.438%)
Epoch: [16][100/391]	Time 0.045 (0.054)	Data 0.002 (0.005)	Loss 0.8239 (0.9606)	Prec 71.094% (66.159%)
Epoch: [16][200/391]	Time 0.052 (0.055)	Data 0.002 (0.003)	Loss 0.9976 (0.9679)	Prec 64.844% (65.718%)
Epoch: [16][300/391]	Time 0.055 (0.055)	Data 0.002 (0.003)	Loss 0.9023 (0.9670)	Prec 66.406% (65.695%)
Validation starts
Test: [0/79]	Time 0.270 (0.270)	Loss 1.1486 (1.1486)	Prec 59.375% (59.375%)
 * Prec 62.500% 
best acc: 62.740000
Epoch: [17][0/391]	Time 0.318 (0.318)	Data 0.251 (0.251)	Loss 0.8179 (0.8179)	Prec 72.656% (72.656%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.051 (0.051)	Data 0.003 (0.003)	Loss 0.6723 (0.7064)	Prec 71.094% (75.337%)
Validation starts
Test: [0/79]	Time 0.269 (0.269)	Loss 0.6962 (0.6962)	Prec 75.781% (75.781%)
 * Prec 74.300% 
best acc: 74.300000
Epoch: [31][0/391]	Time 0.315 (0.315)	Data 0.250 (0.250)	Loss 0.6881 (0.6881)	Prec 78.125% (78.125%)
Epoch: [31][100/391]	Time 0.048 (0.056)	Data 0.002 (0.005)	Loss 0.6263 (0.7065)	Prec 77.344% (75.062%)
Epoch: [31][200/391]	Time 0.057 (0.056)	Data 0.002 (0.004)	Loss 0.7131 (0.7019)	Prec 78.906% (75.198%)
Epoch: [31][300/391]	Time 0.045 (0.054)	Data 0.002 (0.003)	Loss 0.6351 (0.7033)	Prec 79.688% (75.260%)
Validation starts
Test: [0/79]	Time 0.257 (0.257)	Loss 0.6707 (0.6707)	Prec 74.219% (74.219%)
 * Prec 73.950% 
best acc: 74.300000
Epoch: [32][0/391]	Time 0.317 (0.317)	Data 0.259 (0.259)	Loss 0.7291 (0.7291)	Prec 74.219% (74.219%)
Epoch: [32][100/391]	Time 0.044 (0.056)	Data 0.002 (0.005)	Loss 0.5513 (0.7056)	Prec 80.469% (75.294%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.271 (0.271)	Loss 0.7709 (0.7709)	Prec 72.656% (72.656%)
 * Prec 73.440% 
best acc: 74.810000
Epoch: [46][0/391]	Time 0.344 (0.344)	Data 0.278 (0.278)	Loss 0.5986 (0.5986)	Prec 78.906% (78.906%)
Epoch: [46][100/391]	Time 0.067 (0.062)	Data 0.004 (0.005)	Loss 0.6952 (0.6862)	Prec 78.906% (75.897%)
Epoch: [46][200/391]	Time 0.065 (0.061)	Data 0.003 (0.004)	Loss 0.6980 (0.6943)	Prec 76.562% (75.587%)
Epoch: [46][300/391]	Time 0.052 (0.059)	Data 0.002 (0.003)	Loss 0.7904 (0.7022)	Prec 71.875% (75.231%)
Validation starts
Test: [0/79]	Time 0.271 (0.271)	Loss 0.6634 (0.6634)	Prec 73.438% (73.438%)
 * Prec 72.970% 
best acc: 74.810000
Epoch: [47][0/391]	Time 0.347 (0.347)	Data 0.280 (0.280)	Loss 0.7443 (0.7443)	Prec 75.781% (75.781%)
Epoch: [47][100/391]	Time 0.060 (0.063)	Data 0.003 (0.005)	Loss 0.7180 (0.7066)	Prec 75.000% (74.946%)
Epoch: [47][200/391]	Time 0.063 (0.062)	Data 0.002 (0.004)	Loss 0.6282 (0.7010)	Prec 79.688% (75.365%)
Epoch: [47][300/391]	

 * Prec 74.500% 
best acc: 74.810000
Epoch: [61][0/391]	Time 0.331 (0.331)	Data 0.265 (0.265)	Loss 0.6645 (0.6645)	Prec 75.000% (75.000%)
Epoch: [61][100/391]	Time 0.073 (0.063)	Data 0.003 (0.005)	Loss 0.5948 (0.6938)	Prec 77.344% (75.712%)
Epoch: [61][200/391]	Time 0.042 (0.059)	Data 0.002 (0.004)	Loss 0.7424 (0.6959)	Prec 72.656% (75.548%)
Epoch: [61][300/391]	Time 0.053 (0.058)	Data 0.003 (0.003)	Loss 0.8163 (0.7006)	Prec 78.906% (75.236%)
Validation starts
Test: [0/79]	Time 0.257 (0.257)	Loss 0.7376 (0.7376)	Prec 73.438% (73.438%)
 * Prec 74.430% 
best acc: 74.810000
Epoch: [62][0/391]	Time 0.312 (0.312)	Data 0.245 (0.245)	Loss 0.8946 (0.8946)	Prec 72.656% (72.656%)
Epoch: [62][100/391]	Time 0.063 (0.060)	Data 0.003 (0.005)	Loss 0.5776 (0.7105)	Prec 80.469% (74.745%)
Epoch: [62][200/391]	Time 0.058 (0.057)	Data 0.002 (0.004)	Loss 0.6060 (0.7086)	Prec 81.250% (74.973%)
Epoch: [62][300/391]	Time 0.051 (0.056)	Data 0.003 (0.003)	Loss 0.7364 (0.7090)	Prec 71.094% (75.091%)
Validation s

Epoch: [76][100/391]	Time 0.057 (0.051)	Data 0.002 (0.005)	Loss 0.7091 (0.6944)	Prec 74.219% (75.665%)
Epoch: [76][200/391]	Time 0.060 (0.052)	Data 0.003 (0.003)	Loss 0.5973 (0.6968)	Prec 82.031% (75.711%)
Epoch: [76][300/391]	Time 0.056 (0.052)	Data 0.003 (0.003)	Loss 0.8208 (0.7014)	Prec 72.656% (75.496%)
Validation starts
Test: [0/79]	Time 0.247 (0.247)	Loss 0.6731 (0.6731)	Prec 74.219% (74.219%)
 * Prec 73.360% 
best acc: 74.810000
Epoch: [77][0/391]	Time 0.311 (0.311)	Data 0.253 (0.253)	Loss 0.5754 (0.5754)	Prec 79.688% (79.688%)
Epoch: [77][100/391]	Time 0.069 (0.064)	Data 0.003 (0.005)	Loss 0.7999 (0.7126)	Prec 74.219% (75.085%)
Epoch: [77][200/391]	Time 0.054 (0.062)	Data 0.002 (0.004)	Loss 0.7640 (0.7049)	Prec 74.219% (75.210%)
Epoch: [77][300/391]	Time 0.055 (0.061)	Data 0.002 (0.003)	Loss 0.7785 (0.7065)	Prec 71.094% (75.221%)
Validation starts
Test: [0/79]	Time 0.261 (0.261)	Loss 0.6833 (0.6833)	Prec 76.562% (76.562%)
 * Prec 73.760% 
best acc: 74.810000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.058 (0.053)	Data 0.002 (0.004)	Loss 0.7951 (0.6992)	Prec 71.875% (75.610%)
Epoch: [91][300/391]	Time 0.052 (0.056)	Data 0.002 (0.003)	Loss 0.9372 (0.6996)	Prec 66.406% (75.519%)
Validation starts
Test: [0/79]	Time 0.240 (0.240)	Loss 0.6605 (0.6605)	Prec 75.000% (75.000%)
 * Prec 74.010% 
best acc: 74.810000
Epoch: [92][0/391]	Time 0.327 (0.327)	Data 0.260 (0.260)	Loss 0.7414 (0.7414)	Prec 75.781% (75.781%)
Epoch: [92][100/391]	Time 0.064 (0.061)	Data 0.003 (0.005)	Loss 0.6736 (0.7016)	Prec 77.344% (75.487%)
Epoch: [92][200/391]	Time 0.054 (0.059)	Data 0.003 (0.004)	Loss 0.6066 (0.6985)	Prec 82.812% (75.634%)
Epoch: [92][300/391]	Time 0.063 (0.058)	Data 0.003 (0.003)	Loss 0.5696 (0.7037)	Prec 79.688% (75.330%)
Validation starts
Test: [0/79]	Time 0.259 (0.259)	Loss 0.6686 (0.6686)	Prec 71.875% (71.875%)
 * Prec 74.620% 
best acc: 74.810000
Epoch: [93][0/391]	Time 0.335 (0.335)	Data 0.270 (0.270)	Loss 0.6430 (0.6430)	Prec 73.438% (73.438%)
Epoch: [93][100/391]	

In [17]:
PATH = "result/" + model_name + "/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 7481/10000 (75%)



In [18]:
######### Save inputs from selected layer ##########
save_output = SaveOutput()
model.layer1[0].conv2.register_forward_pre_hook(save_output) ## Grab input for BasicBlock0 Conv1 layer
# model.layer1[1].conv1.register_forward_pre_hook(save_output) ## Grab input for BasicBlock1 Conv1 layer
####################################################

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to('cuda')
# Run a forward pass to prehook
out = model(images)

In [19]:
w_bit = 4
weight_q = model.layer1[0].conv2.weight_q # quantized value is stored during the training
# print(weight_q)
w_alpha = model.layer1[0].conv2.weight_quant.wgt_alpha   # alpha is defined in your model already. bring it out here
w_delta = w_alpha/(2**(w_bit-1)-1)   # delta can be calculated by using alpha and w_bit
weight_int = weight_q / w_delta # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-7., -7.,  0.],
          [-7.,  0.,  7.],
          [-0.,  0.,  7.]],

         [[-0.,  0.,  0.],
          [-0., -7.,  0.],
          [ 0.,  0.,  7.]],

         [[-0.,  0.,  0.],
          [-0.,  0.,  7.],
          [-7., -0.,  0.]],

         ...,

         [[ 0., -0.,  7.],
          [-0., -7.,  0.],
          [ 0.,  0., -0.]],

         [[-0., -0.,  0.],
          [-0.,  0.,  7.],
          [ 0., -0.,  0.]],

         [[-0., -0., -7.],
          [-0., -0., -7.],
          [-7., -0., -7.]]],


        [[[ 0.,  0., -7.],
          [ 0., -0., -7.],
          [ 7.,  7.,  0.]],

         [[-7., -0., -0.],
          [-0., -0., -0.],
          [-7., -7., -0.]],

         [[ 7.,  7.,  7.],
          [-0.,  0.,  0.],
          [-7., -7., -0.]],

         ...,

         [[ 0.,  7.,  0.],
          [ 0.,  7., -0.],
          [-0.,  0.,  0.]],

         [[-0.,  0., -0.],
          [-0., -0., -0.],
          [ 0.,  0.,  0.]],

         [[-7., -0., -0.],
          [ 0.,  0.,  7.],
 

In [20]:
x_bit = 4    
x = save_output.outputs[0][0]  # input of the 2nd conv layer
x_alpha = model.layer1[0].conv2.act_alpha
x_delta = x_alpha/(2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
          [ 8.0000, 11.0000, 11.0000,  ...,  6.0000,  2.0000,  5.0000],
          [14.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 1.0000,  0.0000,  0.0000,  ...,  4.0000,  0.0000,  8.0000],
          [ 1.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  6.0000],
          [ 2.0000,  0.0000,  0.0000,  ...,  4.0000,  4.0000,  8.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 4.0000,  4.0000,  4.0000,  ...,  4.0000,  4.0000,  0.0000],
          [10.0000,  8.0000,  8.0000,  ..., 10.0000,  5.0000,  1.0000],
          ...,
          [ 5.0000,  1.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 7.0000,  4.0000,  0.0000,  ...,  0.0000,  1.0000,  0.0000],
          [ 7.0000,  4.0000,  0.0000,  ...,  1.0000,  2.0000,  1.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 4.0000,  7.0000,  

In [21]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_recovered = output_int*x_delta*w_delta  # recover with x_delta and w_delta
print(output_recovered)

tensor([[[[ 7.9777e+01,  7.7303e+01,  7.9777e+01,  ...,  4.5764e+01,
            3.9579e+01,  5.0711e+01],
          [ 4.3290e+01,  5.1329e+01,  5.5040e+01,  ...,  1.9171e+01,
            2.4737e+00,  3.7724e+01],
          [ 5.6895e+01,  7.4830e+01,  8.0395e+01,  ...,  4.0198e+01,
            1.2369e+00,  3.0303e+01],
          ...,
          [-3.1540e+01, -5.2566e+01, -6.1224e+01,  ..., -3.4013e+01,
           -6.1843e+01, -6.8027e+00],
          [-1.4224e+01, -3.0303e+01, -4.2053e+01,  ..., -4.8856e+01,
           -6.5553e+01, -5.5658e+00],
          [-6.3698e+01, -5.5658e+01, -4.0198e+01,  ..., -4.8237e+01,
           -4.4527e+01, -3.0921e+00]],

         [[-8.1632e+01, -8.8435e+01, -9.0290e+01,  ..., -6.1224e+01,
           -4.7619e+01, -2.9066e+01],
          [-9.2764e+01, -9.5238e+01, -9.7093e+01,  ..., -6.4316e+01,
           -4.2671e+01, -1.7934e+01],
          [-1.1070e+02, -1.0822e+02, -1.0884e+02,  ..., -6.8027e+01,
           -3.8342e+01, -2.0408e+01],
          ...,
     

In [22]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv2.weight_q
print(conv_ref.weight)
output_ref = conv_ref(x)
print(output_ref)

Parameter containing:
tensor([[[[-2.3379, -2.3379,  0.0000],
          [-2.3379,  0.0000,  2.3379],
          [-0.0000,  0.0000,  2.3379]],

         [[-0.0000,  0.0000,  0.0000],
          [-0.0000, -2.3379,  0.0000],
          [ 0.0000,  0.0000,  2.3379]],

         [[-0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000,  2.3379],
          [-2.3379, -0.0000,  0.0000]],

         ...,

         [[ 0.0000, -0.0000,  2.3379],
          [-0.0000, -2.3379,  0.0000],
          [ 0.0000,  0.0000, -0.0000]],

         [[-0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  2.3379],
          [ 0.0000, -0.0000,  0.0000]],

         [[-0.0000, -0.0000, -2.3379],
          [-0.0000, -0.0000, -2.3379],
          [-2.3379, -0.0000, -2.3379]]],


        [[[ 0.0000,  0.0000, -2.3379],
          [ 0.0000, -0.0000, -2.3379],
          [ 2.3379,  2.3379,  0.0000]],

         [[-2.3379, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000],
          [-2.3379, -2.3379, -0.0000]],

         

In [23]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.3655, device='cuda:0', grad_fn=<MeanBackward0>)


In [24]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
weight = model.layer1[0].conv2.weight_q
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)

tensor([[[[ 7.9488e+01,  7.2294e+01,  7.3883e+01,  ...,  4.2863e+01,
            3.5962e+01,  4.0543e+01],
          [ 4.2896e+01,  4.5064e+01,  4.7090e+01,  ...,  1.3974e+01,
           -9.1334e-01,  3.2183e+01],
          [ 4.5436e+01,  6.1701e+01,  6.6042e+01,  ...,  3.1512e+01,
           -5.1399e+00,  2.8205e+01],
          ...,
          [-2.6195e+01, -4.4787e+01, -5.1329e+01,  ..., -2.7906e+01,
           -5.0751e+01, -5.3255e+00],
          [-1.1419e+01, -2.5167e+01, -3.5295e+01,  ..., -4.2138e+01,
           -5.5156e+01, -5.0276e+00],
          [-5.3165e+01, -4.5605e+01, -3.4420e+01,  ..., -4.0040e+01,
           -3.7708e+01, -3.0274e+00]],

         [[-7.8098e+01, -8.3322e+01, -8.3932e+01,  ..., -5.4376e+01,
           -3.8674e+01, -2.4085e+01],
          [-7.7606e+01, -7.7313e+01, -7.9159e+01,  ..., -5.1837e+01,
           -3.2733e+01, -1.5501e+01],
          [-8.9581e+01, -8.8128e+01, -8.8525e+01,  ..., -5.4616e+01,
           -2.9570e+01, -1.7681e+01],
          ...,
     

In [25]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(4.9863, device='cuda:0', grad_fn=<MeanBackward0>)
